# **_Exploratory Data Analysis- IPL Data_**

### **I. Importing essential libraries**

In [179]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
%matplotlib inline
import plotly.express as px

____

### **II.Dataset properties**

In [180]:
df = pd.read_csv("deliveries_updated_mens_ipl_upto_2024 - deliveries_updated_mens_ipl_upto_2024.csv")
df.head()

,matchId,inning,over_ball,over,ball,batting_team,bowling_team,batsman,non_striker,bowler,batsman_runs,extras,isWide,isNoBall,Byes,LegByes,Penalty,dismissal_kind,player_dismissed,date
0,335982,1,0.1,0,1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,P Kumar,0,1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2008-04-18
1,335982,1,0.2,0,2,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-04-18
2,335982,1,0.3,0,3,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2008-04-18
3,335982,1,0.4,0,4,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-04-18
4,335982,1,0.5,0,5,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-04-18


In [181]:
df.columns

Index(['matchId', 'inning', 'over_ball', 'over', 'ball', 'batting_team',
       'bowling_team', 'batsman', 'non_striker', 'bowler', 'batsman_runs',
       'extras', 'isWide', 'isNoBall', 'Byes', 'LegByes', 'Penalty',
       'dismissal_kind', 'player_dismissed', 'date'],
      dtype='object')

In [182]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260920 entries, 0 to 260919
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   matchId           260920 non-null  int64  
 1   inning            260920 non-null  int64  
 2   over_ball         260920 non-null  float64
 3   over              260920 non-null  int64  
 4   ball              260920 non-null  int64  
 5   batting_team      260920 non-null  object 
 6   bowling_team      260920 non-null  object 
 7   batsman           260920 non-null  object 
 8   non_striker       260920 non-null  object 
 9   bowler            260920 non-null  object 
 10  batsman_runs      260920 non-null  int64  
 11  extras            260920 non-null  int64  
 12  isWide            8381 non-null    float64
 13  isNoBall          1093 non-null    float64
 14  Byes              673 non-null     float64
 15  LegByes           4001 non-null    float64
 16  Penalty           2 

In [183]:
df.shape

(260920, 20)

In [184]:
df.isnull().sum()

matchId                  0
inning                   0
over_ball                0
over                     0
ball                     0
batting_team             0
bowling_team             0
batsman                  0
non_striker              0
bowler                   0
batsman_runs             0
extras                   0
isWide              252539
isNoBall            259827
Byes                260247
LegByes             256919
Penalty             260918
dismissal_kind      247970
player_dismissed    247970
date                     0
dtype: int64

In [185]:
# For finding all missing values
[i for i in df.columns if df[i].isnull().sum()>0]

['isWide',
 'isNoBall',
 'Byes',
 'LegByes',
 'Penalty',
 'dismissal_kind',
 'player_dismissed']

___

### **III. Data cleaning**
- Handling invalid datatypes


In [186]:
filled_df=df

filled_df.date = pd.to_datetime(filled_df.date,errors="coerce")

> The date column is an **object** datatype, while it should be a **datetime** datatype for analysis

- Dropping unneccesary columns

In [187]:
try:
    filled_df=filled_df.drop(columns="Penalty")
except:
    None

> The Penalty columns has only 2 valid values out of 260920 values (**0.000766518%**), so it can be dropped as it has no significance on the dataset

- Handling Mising values

In [188]:
fillwith0=["isWide","isNoBall","Byes","LegByes",]
filled_df[fillwith0]=filled_df[fillwith0].fillna(0)

fillwithStr = ["dismissal_kind","player_dismissed"]
filled_df[fillwithStr]=filled_df[fillwithStr].fillna("No Dismissal")


>There is a large amount of missing values in the above columns, but these columns can be found useful for finding insights, thus they are filled with appropriate values (The dismissal_kind is used to tally the number of wickets a bowler has taken)

- Feature binning

In [189]:
best_batsman = filled_df.groupby('batsman')['batsman_runs'].sum().sort_values(ascending=False)
best_batsman.head(10)

batsman
V Kohli           8014
S Dhawan          6769
RG Sharma         6630
DA Warner         6567
SK Raina          5536
MS Dhoni          5243
AB de Villiers    5181
CH Gayle          4997
RV Uthappa        4954
KD Karthik        4843
Name: batsman_runs, dtype: int64

In [190]:
filled_df.dismissal_kind.value_counts()

dismissal_kind
No Dismissal             247970
caught                     8063
bowled                     2212
run out                    1114
lbw                         800
caught and bowled           367
stumped                     358
retired hurt                 15
hit wicket                   15
obstructing the field         3
retired out                   3
Name: count, dtype: int64

In [191]:
wickets = filled_df[~filled_df['dismissal_kind'].isin(['run out', 'retired hurt', 'obstructing the field','No Dismissal'])]
filled_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260920 entries, 0 to 260919
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   matchId           260920 non-null  int64         
 1   inning            260920 non-null  int64         
 2   over_ball         260920 non-null  float64       
 3   over              260920 non-null  int64         
 4   ball              260920 non-null  int64         
 5   batting_team      260920 non-null  object        
 6   bowling_team      260920 non-null  object        
 7   batsman           260920 non-null  object        
 8   non_striker       260920 non-null  object        
 9   bowler            260920 non-null  object        
 10  batsman_runs      260920 non-null  int64         
 11  extras            260920 non-null  int64         
 12  isWide            260920 non-null  float64       
 13  isNoBall          260920 non-null  float64       
 14  Byes

> Dismissal by **run out, retired/hurt, obstructing the field** are the types of wickets not credited to the bowler, thus they are excluded

In [192]:
best_bowler = wickets.groupby('bowler')['dismissal_kind'].count().sort_values(ascending=False)
best_bowler.head(10)

bowler
YS Chahal     205
PP Chawla     192
DJ Bravo      183
B Kumar       181
SP Narine     180
R Ashwin      180
A Mishra      174
SL Malinga    170
JJ Bumrah     168
RA Jadeja     160
Name: dismissal_kind, dtype: int64

In [193]:
def gen_phase(over):
    if over < 6:
        return 'powerplay'
    elif over < 16:
        return 'middle'
    else:
        return 'death'

filled_df['phase']= filled_df['over'].apply(gen_phase)


> Deriving a new column **Phase** to group overs by their respective phase and collect important insights

In [194]:
death_overdf = filled_df[filled_df['phase']=='death']
batsman_stat = death_overdf.groupby('batsman').agg({
    'batsman_runs': 'sum',
    'ball': 'count'
})

batsman_stat = batsman_stat[batsman_stat['ball']>=200]
batsman_stat['strike_rate'] = (batsman_stat['batsman_runs']/batsman_stat['ball'])*100

top_deathover_batsman = batsman_stat.sort_values(by='strike_rate',ascending = False)
top_deathover_batsman.head(10)

,batsman_runs,ball,strike_rate
batsman,,,
AB de Villiers,1421,635,223.779528
RR Pant,626,318,196.855346
CH Gayle,404,209,193.301435
V Kohli,1099,571,192.469352
F du Plessis,416,220,189.090909
RG Sharma,1176,625,188.160000
SA Yadav,516,276,186.956522
AD Russell,1065,570,186.842105
TH David,452,244,185.245902


___

### **IV.Graph Plotting**

In [195]:

best_batsmen_df = best_batsman.reset_index()

fig_batsman = px.bar(
    best_batsmen_df,
    x='batsman',
    y='batsman_runs',
    title='Top 10 Run Scorers in IPL History',
    labels={'batsman': 'Batsman', 'batsman_runs': 'Total Runs'},
    color='batsman',
    template="plotly_dark"
)

fig_batsman.update_layout(
    xaxis_title="Batsman",
    yaxis_title="Runs Scored",
    font=dict(family="Arial, sans-serif", size=12)
)



fig_batsman.show()

fig.write_html("Top 10 Run Scorers in IPL History.html")


In [196]:

top_bowlers_df = best_bowler.reset_index()


fig_bowlers = px.bar(
    top_bowlers_df,
    x='bowler',
    y='dismissal_kind',
    title='Top 10 Wicket-Takers in IPL History',
    labels={'bowler': 'Bowler', 'dismissal_kind': 'Total Wickets'},
    color='bowler',  
    template='plotly_dark'
)

fig_bowlers.update_layout(
    xaxis_title="Bowler",
    yaxis_title="Total Wickets",
    font=dict(family="Arial, sans-serif", size=12)
)


fig_bowlers.show()
fig_bowlers.write_html("Top 10 Wicket-Takers in IPL History.html")


In [197]:
best_deathover_df = top_deathover_batsman.reset_index()


fig_deathover = px.bar(
    best_deathover_df,
    x='batsman',
    y='strike_rate',
    title='Most Destructive Death-Over Batsmen (min. 200 balls)',
    labels={'batsman': 'Batsman', 'strike_rate': 'Strike-Rate'},
    color='strike_rate',  
    template='plotly_dark'
)

fig_bowlers.update_layout(
    xaxis_title="Batsman",
    yaxis_title="Strike-Rate (Runs per 100 balls)",
    font=dict(family="Arial, sans-serif", size=12)
)


fig_deathover.show()
fig_deathover.write_html("Most Destructive Death-Over Batsmen.html")


### **V.Insights**

**Insight 1: YS Chahal is the All-Time Wicket King.**

Finding: Yuzvendra Chahal has the most wickets in IPL history.

Actionable Insight: When a team is in desperate need of a breakthrough, especially in the middle overs where Chahal thrives, he is statistically the most reliable bowler in the league's history to get a wicket. Teams should consider using an extra spinner against oppositions that are weak against leg-spin.

_______

**Insight 2: Andre Russell is a Death-Over Specialist.**

Finding: While AB de Villiers has a higher strike rate, Andre Russell has scored almost double the runs at a phenomenal strike rate in the death overs.

Teams playing against KKR must have a clear plan for Russell in the final 4-5 overs. Bowling wide yorkers or using their most experienced death bowler against him is crucial. For KKR, ensuring Russell faces as many balls as possible in this phase is a key to winning games.


________

**Insight 3: Virat Kohli's Game Has Evolved.**

Finding: Virat Kohli is the highest run-scorer overall, and he also appears in the top 5 for most destructive death-overs batsmen.

Traditionally known as an accumulator, Kohli has transformed his game to become a powerful finisher. This dual ability makes him uniquely dangerous. Opposition can no longer relax once he is 'set'; they need a specific plan to contain his explosive late-game hitting.